<a href="https://colab.research.google.com/github/stazam/M7DataSp---Advanced-data-science-techniques/blob/main/HocekyBetting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [185]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split

In [186]:
from google.colab import files
uploaded = files.upload()

Saving hockey_datas.csv to hockey_datas (3).csv


In [187]:
df = pd.read_csv("hockey_datas.csv")

In [188]:
df.shape

(1396, 23)

In [189]:
df.head()

,Unnamed: 0,id,game_number,date,home,away,place,viewers,win_periods_home,draw_periods,win_periods_away,score_home,score_away,shots_home,shots_away,saves_home,saves_away,powerplay_home,powerplay_away,weakening_home,weakening_away,faceoff_home,faceoff_away
0,0,83430,1,11/09/2015,"HK Poprad, s.r.o.",HKM a.s. Zvolen,ZŠ Poprad,1049,2,0,1,3,2,0.1000,0.0769,0.9231,0.9000,0.3333,0.0000,1.0000,0.6667,0.0,0.0
1,1,83431,2,11/09/2015,"HK DUKLA Trenčín, a.s.",MHC MOUNTFIELD Martin,"ZŠ Pavla Demitru, Trenčín",1630,2,1,0,3,1,0.0732,0.0294,0.9706,0.9268,0.5000,0.0000,1.0000,0.5000,0.0,0.0
2,2,83432,3,11/09/2015,"HK 36 Skalica, s.r.o.","ŠHK 37 Piešťany, s.r.o.",HANT arena Skalica,1086,0,2,1,3,5,0.0938,0.1190,0.8810,0.9063,0.0000,0.3333,0.6667,1.0000,0.0,0.0
3,3,83433,4,11/09/2015,HC Košice s.r.o.,MsHK DOXXbet Žilina,STEEL Arena Košice,2548,2,1,1,5,4,0.1250,0.1667,0.8333,0.8750,0.2000,0.7500,0.2500,0.8000,0.0,0.0
4,4,83434,5,10/09/2015,HC ‘05 iClinic BANSKÁ BYSTRICA,"HK Nitra, a.s.",ZŠ Banská Bystrica,1896,2,0,1,4,1,0.1081,0.0400,0.9600,0.8919,0.0000,0.0000,1.0000,1.0000,0.0,0.0


1. pridat stplpec result - HOTOVO
2. zmeit stlpec date na datetime premennu
3. upravit vsetky hodnoty v stlpcoch
4. pridat sltpce s  premennymi ktore mal ten typek
6. pozot na chybajuce pozorovania pri faceoff, places a skontorlovat aj dalsie a upravit a nahradit medianom alebo takym niecim
7. zistit unikatne hodnoty a podla toho to upravit

conceded = pocet inkasovanych golov

H2H v specifickej sezone a sucasne aj za celkove predchadzajuce obdobie

vitazstva celkovo za vsetky sezony, vitazstva celkovo za danu sezonu proti danemu superovi

urcite treba pridat aj priemrene buly protis uperovi, to je podstatne!!!!

In [190]:
def name_parser(x):

  x = x.lower()
  clubs = ['Poprad', 'Trenčín', 'Skalica', 'Košice', 'BANSKÁ BYSTRICA', 'Zvolen', 'Piešťany', 'Martin', 'Nitra', 'Žilina', 
           'Liptovský Mikuláš', 'Nové Zámky', 'Detva', 'Újbuda', 'Miskolc', 'Michalovce', 'Bratislava']

  clubs = [x.lower() for x in clubs]
  for club in clubs:
    if club in x:
      return club
  
  return 

df['home'] = df['home'].apply(name_parser)
df['away'] = df['away'].apply(name_parser)

In [191]:
#mozno by som este mohol pridat to, ze ked nie je domaci tym na domacom lade, tak ako sa to premietne do vysledku -- ale to sa stava v minime pripadov
import datetime

df['date'] = pd.to_datetime(df['date'],format = "%d/%m/%Y")


In [192]:
df.loc[df['score_home'] > df['score_away'], 'result'] = 'home'
df.loc[df['score_home'] < df['score_away'], 'result'] = 'away'
df.loc[df['win_periods_home'] + df['draw_periods'] + df['win_periods_away'] >= 4, 'result'] = 'draw'

In [193]:
def season(x):

  if x >= 83430 and x <= 83699:
    return "2015/2016"
  elif x >= 87717 and x <= 87986:
    return "2016/2017"
  elif x >= 96815 and x <= 97084:
    return "2017/2018"
  elif x >= 101458 and x <= 101787:
    return "2018/2019"
  else:
    return "2019/2020"

df['season'] = df['id'].apply(season)

In [194]:
df

,Unnamed: 0,id,game_number,date,home,away,place,viewers,win_periods_home,draw_periods,win_periods_away,score_home,score_away,shots_home,shots_away,saves_home,saves_away,powerplay_home,powerplay_away,weakening_home,weakening_away,faceoff_home,faceoff_away,result,season
0,0,83430,1,2015-09-11,poprad,zvolen,ZŠ Poprad,1049,2,0,1,3,2,0.1000,0.0769,0.9231,0.9000,0.3333,0.0000,1.0000,0.6667,0.0000,0.0000,home,2015/2016
1,1,83431,2,2015-09-11,trenčín,martin,"ZŠ Pavla Demitru, Trenčín",1630,2,1,0,3,1,0.0732,0.0294,0.9706,0.9268,0.5000,0.0000,1.0000,0.5000,0.0000,0.0000,home,2015/2016
2,2,83432,3,2015-09-11,skalica,piešťany,HANT arena Skalica,1086,0,2,1,3,5,0.0938,0.1190,0.8810,0.9063,0.0000,0.3333,0.6667,1.0000,0.0000,0.0000,away,2015/2016
3,3,83433,4,2015-09-11,košice,žilina,STEEL Arena Košice,2548,2,1,1,5,4,0.1250,0.1667,0.8333,0.8750,0.2000,0.7500,0.2500,0.8000,0.0000,0.0000,draw,2015/2016
4,4,83434,5,2015-09-10,banská bystrica,nitra,ZŠ Banská Bystrica,1896,2,0,1,4,1,0.1081,0.0400,0.9600,0.8919,0.0000,0.0000,1.0000,1.0000,0.0000,0.0000,home,2015/2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1391,1391,110103,358,2020-02-21,miskolc,újbuda,"Ice rink Miskolc, HUN",1385,1,0,2,1,3,0.0222,0.1000,0.9259,0.9778,0.0000,0.2500,0.7500,1.0000,0.5574,0.4426,away,2019/2020
1392,1392,110105,360,2020-02-21,košice,nové zámky,STEEL Arena Košice,7067,2,1,0,7,2,0.1892,0.0952,0.9048,0.8108,0.1429,0.2500,0.7500,0.8571,0.5536,0.4464,home,2019/2020
1393,1393,110106,361,2020-02-21,trenčín,poprad,"ZŠ Pavla Demitru, Trenčín",3120,1,0,3,3,4,0.0000,0.1212,0.8788,0.9143,0.0000,0.0000,1.0000,1.0000,0.0000,0.0000,draw,2019/2020
1394,1394,110107,362,2020-02-21,bratislava,liptovský mikuláš,Arena Ondreja Nepelu,4697,3,0,0,11,0,0.2500,0.0000,1.0000,0.7500,0.5000,0.0000,1.0000,0.5000,0.5893,0.4107,home,2019/2020


In [195]:
df_home = df.copy()  
df_away = df.copy()

In [196]:
df_home.columns = df_home.columns.str.replace("home", "team")
df_home.columns = df_home.columns.str.replace("away", "opponent")
df_home = df_home.replace('home', 'win')
df_home = df_home.replace('away', 'lost')
df_home['h_a'] = 'home'

In [197]:
df_home

,Unnamed: 0,id,game_number,date,team,opponent,place,viewers,win_periods_team,draw_periods,win_periods_opponent,score_team,score_opponent,shots_team,shots_opponent,saves_team,saves_opponent,powerplay_team,powerplay_opponent,weakening_team,weakening_opponent,faceoff_team,faceoff_opponent,result,season,h_a
0,0,83430,1,2015-09-11,poprad,zvolen,ZŠ Poprad,1049,2,0,1,3,2,0.1000,0.0769,0.9231,0.9000,0.3333,0.0000,1.0000,0.6667,0.0000,0.0000,win,2015/2016,home
1,1,83431,2,2015-09-11,trenčín,martin,"ZŠ Pavla Demitru, Trenčín",1630,2,1,0,3,1,0.0732,0.0294,0.9706,0.9268,0.5000,0.0000,1.0000,0.5000,0.0000,0.0000,win,2015/2016,home
2,2,83432,3,2015-09-11,skalica,piešťany,HANT arena Skalica,1086,0,2,1,3,5,0.0938,0.1190,0.8810,0.9063,0.0000,0.3333,0.6667,1.0000,0.0000,0.0000,lost,2015/2016,home
3,3,83433,4,2015-09-11,košice,žilina,STEEL Arena Košice,2548,2,1,1,5,4,0.1250,0.1667,0.8333,0.8750,0.2000,0.7500,0.2500,0.8000,0.0000,0.0000,draw,2015/2016,home
4,4,83434,5,2015-09-10,banská bystrica,nitra,ZŠ Banská Bystrica,1896,2,0,1,4,1,0.1081,0.0400,0.9600,0.8919,0.0000,0.0000,1.0000,1.0000,0.0000,0.0000,win,2015/2016,home
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1391,1391,110103,358,2020-02-21,miskolc,újbuda,"Ice rink Miskolc, HUN",1385,1,0,2,1,3,0.0222,0.1000,0.9259,0.9778,0.0000,0.2500,0.7500,1.0000,0.5574,0.4426,lost,2019/2020,home
1392,1392,110105,360,2020-02-21,košice,nové zámky,STEEL Arena Košice,7067,2,1,0,7,2,0.1892,0.0952,0.9048,0.8108,0.1429,0.2500,0.7500,0.8571,0.5536,0.4464,win,2019/2020,home
1393,1393,110106,361,2020-02-21,trenčín,poprad,"ZŠ Pavla Demitru, Trenčín",3120,1,0,3,3,4,0.0000,0.1212,0.8788,0.9143,0.0000,0.0000,1.0000,1.0000,0.0000,0.0000,draw,2019/2020,home
1394,1394,110107,362,2020-02-21,bratislava,liptovský mikuláš,Arena Ondreja Nepelu,4697,3,0,0,11,0,0.2500,0.0000,1.0000,0.7500,0.5000,0.0000,1.0000,0.5000,0.5893,0.4107,win,2019/2020,home


In [198]:
df_away.columns = df_away.columns.str.replace("away", "team")
df_away.columns = df_away.columns.str.replace("home", "opponent")
df_away = df_away.replace('away', 'win')
df_away = df_away.replace('home', 'lost')
df_away['h_a'] = 'away'

In [199]:
df_home.dtypes

Unnamed: 0                       int64
id                               int64
game_number                      int64
date                    datetime64[ns]
team                            object
opponent                        object
place                           object
viewers                          int64
win_periods_team                 int64
draw_periods                     int64
win_periods_opponent             int64
score_team                       int64
score_opponent                   int64
shots_team                     float64
shots_opponent                 float64
saves_team                     float64
saves_opponent                 float64
powerplay_team                 float64
powerplay_opponent             float64
weakening_team                 float64
weakening_opponent             float64
faceoff_team                   float64
faceoff_opponent               float64
result                          object
season                          object
h_a                      

In [200]:
df_teams = pd.concat([df_home, df_away]).reset_index(drop=True)

In [201]:
 df_teams = df_teams.sort_values(['team', 'date']).reset_index(drop=True)

In [202]:
df_teams['win'] = 0
df_teams.loc[df_teams['result'] == 'win', 'win'] = 1
df_teams['draw'] = 0
df_teams.loc[df_teams['result'] == 'draw', 'draw'] = 1
df_teams['lost'] = 0
df_teams.loc[df_teams['result'] == 'lost', 'lost'] = 1   

In [203]:
#groupby - zgrupovanie odpvedajucich si dat, rolling - beriem iba cast tych dat, na ktore beriem priemer, shift - posunutie tych hodnot v ramci tych zgrupovanych skupin
df_teams['win_R5'] = df_teams.groupby(['team', 'season'])['win'].rolling(5, min_periods=1).mean().values
df_teams['win_R5'] =df_teams.groupby(['team', 'season'])['win_R5'].shift(fill_value=0)

In [204]:
df_teams['draw_R5'] = df_teams.groupby(['team', 'season'])['draw'].rolling(5, min_periods=1).mean().values
df_teams['draw_R5'] =df_teams.groupby(['team', 'season'])['draw_R5'].shift(fill_value=0)


In [205]:
df_teams['lost_R5'] = df_teams.groupby(['team', 'season'])['lost'].rolling(5, min_periods=1).mean().values
df_teams['lost_R5'] =df_teams.groupby(['team', 'season'])['lost_R5'].shift(fill_value=0)

In [206]:
df_teams['scored_avg_R5'] = df_teams.groupby(['team', 'season'])['score_team'].rolling(5, min_periods=1).mean().values
df_teams['scored_avg_R5'] =df_teams.groupby(['team', 'season'])['scored_avg_R5'].shift(fill_value=0)

df_teams['conceded_avg_R5'] = df_teams.groupby(['team', 'season'])['score_opponent'].rolling(5, min_periods=1).mean().values
df_teams['conceded_avg_R5'] =df_teams.groupby(['team', 'season'])['conceded_avg_R5'].shift(fill_value=0)

In [207]:
df_teams['matchday'] = df_teams.groupby(['team', 'season']).cumcount()+1
df_teams['matchday_h_a'] = df_teams.groupby(['team', 'season', 'h_a']).cumcount() +1 

In [208]:
df_teams['scored_cum'] = df_teams.groupby(['team', 'season'])['score_team'].cumsum()
df_teams['scored_cum_prev'] =df_teams.groupby(['team', 'season'])['scored_cum'].shift(fill_value=0)
df_teams['scored_cum_prev_avg'] = df_teams['scored_cum_prev'] / (df_teams['matchday']-1)
    

In [209]:
df_teams['conceded_cum'] = df_teams.groupby(['team', 'season'])['score_opponent'].cumsum()
df_teams['conceded_cum_prev'] =df_teams.groupby(['team', 'season'])['conceded_cum'].shift(fill_value=0)
df_teams['conceded_cum_prev_avg'] = df_teams['conceded_cum_prev'] / (df_teams['matchday']-1)

In [210]:
df_teams['H2H_W'] = df_teams.groupby(['team', 'opponent'])['win'].cumsum()
df_teams['H2H_W'] =df_teams.groupby(['team', 'opponent'])['H2H_W'].shift(fill_value=0)

In [211]:
df_teams['H2H_D'] = df_teams.groupby(['team', 'opponent'])['draw'].cumsum()
df_teams['H2H_D'] =df_teams.groupby(['team', 'opponent'])['H2H_D'].shift(fill_value=0)

In [212]:
df_teams['H2H_L'] = df_teams.groupby(['team', 'opponent'])['lost'].cumsum()
df_teams['H2H_L'] =df_teams.groupby(['team', 'opponent'])['H2H_L'].shift(fill_value=0)

In [213]:
df_teams[['H2H_W', 'H2H_D', 'H2H_L']] =df_teams[['H2H_W', 'H2H_D', 'H2H_L']].div(df_teams[['H2H_W', 'H2H_D', 'H2H_L']].sum(axis=1),axis=0).fillna(0)

In [214]:
df_teams

,Unnamed: 0,id,game_number,date,team,opponent,place,viewers,win_periods_team,draw_periods,win_periods_opponent,score_team,score_opponent,shots_team,shots_opponent,saves_team,saves_opponent,powerplay_team,powerplay_opponent,weakening_team,weakening_opponent,faceoff_team,faceoff_opponent,result,season,h_a,win,draw,lost,win_R5,draw_R5,lost_R5,scored_avg_R5,conceded_avg_R5,matchday,matchday_h_a,scored_cum,scored_cum_prev,scored_cum_prev_avg,conceded_cum,conceded_cum_prev,conceded_cum_prev_avg,H2H_W,H2H_D,H2H_L
0,4,83434,5,2015-09-10,banská bystrica,nitra,ZŠ Banská Bystrica,1896,2,0,1,4,1,0.1081,0.0400,0.9600,0.8919,0.0000,0.0000,1.0000,1.0000,0.0000,0.0000,win,2015/2016,home,1,0,0,0.00,0.00,0.0,0.000000,0.000000,1,1,4,0,NaN,1,0,NaN,0.000000,0.000000,0.000000
1,6,83436,7,2015-09-13,banská bystrica,košice,STEEL Arena Košice,2741,1,2,0,1,0,0.0500,0.0000,1.0000,0.9500,0.0000,0.0000,1.0000,1.0000,0.0000,0.0000,win,2015/2016,away,1,0,0,1.00,0.00,0.0,4.000000,1.000000,2,1,5,4,4.000000,1,1,1.000000,0.000000,0.000000,0.000000
2,13,83443,14,2015-09-15,banská bystrica,piešťany,ZŠ Banská Bystrica,1973,1,1,1,2,1,0.0513,0.0333,0.9643,0.9512,0.0000,0.0000,1.0000,1.0000,0.0000,0.0000,win,2015/2016,home,1,0,0,1.00,0.00,0.0,2.500000,0.500000,3,2,7,5,2.500000,2,1,0.500000,0.000000,0.000000,0.000000
3,17,83447,18,2015-09-18,banská bystrica,martin,ZŠ Martin,1700,1,2,1,1,2,0.0278,0.0714,0.9286,0.9722,0.0000,0.0000,1.0000,1.0000,0.0000,0.0000,draw,2015/2016,away,0,1,0,1.00,0.00,0.0,2.333333,0.666667,4,2,8,7,2.333333,4,2,0.666667,0.000000,0.000000,0.000000
4,22,83452,23,2015-09-20,banská bystrica,poprad,ZŠ Banská Bystrica,2058,1,3,0,1,0,0.0286,0.0000,1.0000,0.9714,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,draw,2015/2016,home,0,1,0,0.75,0.25,0.0,2.000000,1.000000,5,3,9,8,2.000000,4,4,1.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2787,1057,101759,302,2019-02-15,žilina,újbuda,ZŠ Žilina,581,2,0,1,3,2,0.0732,0.0541,0.9459,0.9268,0.0000,0.3333,0.6667,1.0000,0.4576,0.5424,win,2018/2019,home,1,0,0,0.00,0.40,0.6,2.400000,4.800000,51,26,130,127,2.540000,199,197,3.940000,0.500000,0.250000,0.250000
2788,1067,101769,312,2019-02-17,žilina,nitra,Nitra Aréna,1993,1,0,2,1,3,0.0476,0.0508,0.9655,0.9524,0.1667,0.2500,0.7500,0.8333,0.2273,0.7727,lost,2018/2019,away,0,0,1,0.20,0.40,0.4,2.200000,4.000000,52,26,131,130,2.549020,202,199,3.901961,0.285714,0.047619,0.666667
2789,1068,101770,313,2019-02-22,žilina,liptovský mikuláš,ZŠ Žilina,842,3,0,0,6,1,0.2000,0.0270,0.9730,0.8000,0.0000,0.0000,1.0000,1.0000,0.4800,0.5200,win,2018/2019,home,1,0,0,0.20,0.40,0.4,2.400000,3.600000,53,27,137,131,2.519231,203,202,3.884615,0.466667,0.133333,0.400000
2790,1079,101781,324,2019-02-24,žilina,košice,ZŠ Žilina,815,0,0,3,1,7,0.0333,0.1707,0.8293,0.9667,0.0000,0.2500,0.7500,1.0000,0.5472,0.4528,lost,2018/2019,home,0,0,1,0.40,0.20,0.4,2.800000,2.800000,54,28,138,137,2.584906,210,203,3.830189,0.142857,0.285714,0.571429


In [215]:
df_teams['Num_of_periods'] = df_teams[['win_periods_team','draw_periods','win_periods_opponent']].sum(axis = 1).values
df_teams['points'] = 0

df_teams.loc[ (df_teams['score_team'] > df_teams['score_opponent']) & (df_teams['Num_of_periods'] == 3) ,'points'] = 3 
df_teams.loc[ (df_teams['score_team'] < df_teams['score_opponent']) & (df_teams['Num_of_periods'] == 3) ,'points'] = 0 
df_teams.loc[ (df_teams['score_team'] > df_teams['score_opponent']) & (df_teams['Num_of_periods'] == 4) ,'points'] = 2 
df_teams.loc[ (df_teams['score_team'] < df_teams['score_opponent']) & (df_teams['Num_of_periods'] == 4) ,'points'] = 1 

In [216]:
df_teams['points_cum'] = df_teams.groupby(['team', 'season'])['points'].cumsum()
df_teams['points_cum_prev'] =df_teams.groupby(['team', 'season'])['points_cum'].shift(fill_value=0)    
df_teams['points_cum_prev_avg'] = df_teams['points_cum_prev'] / (df_teams['matchday']-1)

In [217]:
 
df_teams['points_cum_h_a'] = df_teams.groupby(['team', 'season', 'h_a'])['points'].cumsum()
df_teams['points_cum_h_a_prev'] =df_teams.groupby(['team', 'season', 'h_a'])['points_cum_h_a'].shift(fill_value=0)
df_teams['points_cum_h_a_prev_avg'] = df_teams['points_cum_h_a_prev'] / (df_teams['matchday_h_a']-1)
    


In [218]:
df_teams

,Unnamed: 0,id,game_number,date,team,opponent,place,viewers,win_periods_team,draw_periods,win_periods_opponent,score_team,score_opponent,shots_team,shots_opponent,saves_team,saves_opponent,powerplay_team,powerplay_opponent,weakening_team,weakening_opponent,faceoff_team,faceoff_opponent,result,season,h_a,win,draw,lost,win_R5,draw_R5,lost_R5,scored_avg_R5,conceded_avg_R5,matchday,matchday_h_a,scored_cum,scored_cum_prev,scored_cum_prev_avg,conceded_cum,conceded_cum_prev,conceded_cum_prev_avg,H2H_W,H2H_D,H2H_L,Num_of_periods,points,points_cum,points_cum_prev,points_cum_prev_avg,points_cum_h_a,points_cum_h_a_prev,points_cum_h_a_prev_avg
0,4,83434,5,2015-09-10,banská bystrica,nitra,ZŠ Banská Bystrica,1896,2,0,1,4,1,0.1081,0.0400,0.9600,0.8919,0.0000,0.0000,1.0000,1.0000,0.0000,0.0000,win,2015/2016,home,1,0,0,0.00,0.00,0.0,0.000000,0.000000,1,1,4,0,NaN,1,0,NaN,0.000000,0.000000,0.000000,3,3,3,0,NaN,3,0,NaN
1,6,83436,7,2015-09-13,banská bystrica,košice,STEEL Arena Košice,2741,1,2,0,1,0,0.0500,0.0000,1.0000,0.9500,0.0000,0.0000,1.0000,1.0000,0.0000,0.0000,win,2015/2016,away,1,0,0,1.00,0.00,0.0,4.000000,1.000000,2,1,5,4,4.000000,1,1,1.000000,0.000000,0.000000,0.000000,3,3,6,3,3.000000,3,0,NaN
2,13,83443,14,2015-09-15,banská bystrica,piešťany,ZŠ Banská Bystrica,1973,1,1,1,2,1,0.0513,0.0333,0.9643,0.9512,0.0000,0.0000,1.0000,1.0000,0.0000,0.0000,win,2015/2016,home,1,0,0,1.00,0.00,0.0,2.500000,0.500000,3,2,7,5,2.500000,2,1,0.500000,0.000000,0.000000,0.000000,3,3,9,6,3.000000,6,3,3.000000
3,17,83447,18,2015-09-18,banská bystrica,martin,ZŠ Martin,1700,1,2,1,1,2,0.0278,0.0714,0.9286,0.9722,0.0000,0.0000,1.0000,1.0000,0.0000,0.0000,draw,2015/2016,away,0,1,0,1.00,0.00,0.0,2.333333,0.666667,4,2,8,7,2.333333,4,2,0.666667,0.000000,0.000000,0.000000,4,1,10,9,3.000000,4,3,3.000000
4,22,83452,23,2015-09-20,banská bystrica,poprad,ZŠ Banská Bystrica,2058,1,3,0,1,0,0.0286,0.0000,1.0000,0.9714,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,draw,2015/2016,home,0,1,0,0.75,0.25,0.0,2.000000,1.000000,5,3,9,8,2.000000,4,4,1.000000,0.000000,0.000000,0.000000,4,2,12,10,2.500000,8,6,3.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2787,1057,101759,302,2019-02-15,žilina,újbuda,ZŠ Žilina,581,2,0,1,3,2,0.0732,0.0541,0.9459,0.9268,0.0000,0.3333,0.6667,1.0000,0.4576,0.5424,win,2018/2019,home,1,0,0,0.00,0.40,0.6,2.400000,4.800000,51,26,130,127,2.540000,199,197,3.940000,0.500000,0.250000,0.250000,3,3,43,40,0.800000,34,31,1.240000
2788,1067,101769,312,2019-02-17,žilina,nitra,Nitra Aréna,1993,1,0,2,1,3,0.0476,0.0508,0.9655,0.9524,0.1667,0.2500,0.7500,0.8333,0.2273,0.7727,lost,2018/2019,away,0,0,1,0.20,0.40,0.4,2.200000,4.000000,52,26,131,130,2.549020,202,199,3.901961,0.285714,0.047619,0.666667,3,0,43,43,0.843137,9,9,0.360000
2789,1068,101770,313,2019-02-22,žilina,liptovský mikuláš,ZŠ Žilina,842,3,0,0,6,1,0.2000,0.0270,0.9730,0.8000,0.0000,0.0000,1.0000,1.0000,0.4800,0.5200,win,2018/2019,home,1,0,0,0.20,0.40,0.4,2.400000,3.600000,53,27,137,131,2.519231,203,202,3.884615,0.466667,0.133333,0.400000,3,3,46,43,0.826923,37,34,1.307692
2790,1079,101781,324,2019-02-24,žilina,košice,ZŠ Žilina,815,0,0,3,1,7,0.0333,0.1707,0.8293,0.9667,0.0000,0.2500,0.7500,1.0000,0.5472,0.4528,lost,2018/2019,home,0,0,1,0.40,0.20,0.4,2.800000,2.800000,54,28,138,137,2.584906,210,203,3.830189,0.142857,0.285714,0.571429,3,0,46,46,0.867925,37,37,1.370370


In [219]:
df['score'] = df['score_home'].astype(str) + "-" + df['score_away'].astype(str)
df['game'] = df['home'].astype(str) + "-" + df['away'].astype(str)

df_games = df[['date','game', 'score', 'season', 'id', 
                'home', 'away', 'score_home', 'score_away', 'result']]

In [220]:
df_teams = df_teams[['id', 'team', 'h_a', 
                     'win_R5', 'draw_R5', 'lost_R5',  
                     'H2H_W', 'H2H_D', 'H2H_L',
                    'scored_avg_R5', 'conceded_avg_R5',
                    'points_cum_prev_avg', 'points_cum_h_a_prev_avg']]
df_teams.head()

,id,team,h_a,win_R5,draw_R5,lost_R5,H2H_W,H2H_D,H2H_L,scored_avg_R5,conceded_avg_R5,points_cum_prev_avg,points_cum_h_a_prev_avg
0,83434,banská bystrica,home,0.00,0.00,0.0,0.0,0.0,0.0,0.000000,0.000000,NaN,NaN
1,83436,banská bystrica,away,1.00,0.00,0.0,0.0,0.0,0.0,4.000000,1.000000,3.0,NaN
2,83443,banská bystrica,home,1.00,0.00,0.0,0.0,0.0,0.0,2.500000,0.500000,3.0,3.0
3,83447,banská bystrica,away,1.00,0.00,0.0,0.0,0.0,0.0,2.333333,0.666667,3.0,3.0
4,83452,banská bystrica,home,0.75,0.25,0.0,0.0,0.0,0.0,2.000000,1.000000,2.5,3.0


In [221]:
df_home = df_teams[df_teams['h_a']=='home'].copy().drop(['team', 'h_a'], axis=1).add_suffix('_home').rename(columns={"id_home": "id"})
df_away = df_teams[df_teams['h_a']=='away'].copy().drop(['team', 'h_a'], axis=1).add_suffix('_away').rename(columns={"id_away": "id"})

In [222]:
df_games_trend = pd.merge(df_games, df_home, 
                   on='id', how='left')

df_games_trend = pd.merge(df_games_trend, df_away, 
                   on='id', how='left')

In [223]:
df_games_trend=df_games_trend.drop(['H2H_W_away', 'H2H_D_away', 'H2H_L_away'], axis=1)

df_games_trend=df_games_trend.rename(columns={"H2H_W_home": "H2H_W", "H2H_D_home": "H2H_D", "H2H_L_home": "H2H_L"})

In [224]:
df_model = df_games_trend[~df_games_trend.isnull().any(axis=1)].reset_index(drop=True)

df_2019 = df_model[df_model['season']=='2019/2020']
df_model = df_model[df_model['season']!='2019/2020'].reset_index(drop=True)

print("Data current season: " + str(df_2019.shape[0]))
print("Data for modeling: " + str(df_model.shape[0]))

Data current season: 295
Data for modeling: 1030


In [225]:
mapping = {'home': 0, 'draw': 1, 'away': 2}
df_model = df_model.replace({'result': mapping})


In [226]:
df_model.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 27 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   date                          1030 non-null   datetime64[ns]
 1   game                          1030 non-null   object        
 2   score                         1030 non-null   object        
 3   season                        1030 non-null   object        
 4   id                            1030 non-null   int64         
 5   home                          1030 non-null   object        
 6   away                          1030 non-null   object        
 7   score_home                    1030 non-null   int64         
 8   score_away                    1030 non-null   int64         
 9   result                        1030 non-null   int64         
 10  win_R5_home                   1030 non-null   float64       
 11  draw_R5_home                  

In [233]:
X = df_model.iloc[:,10:].values
y = df_model.iloc[:,9].values
info = df_model.iloc[:,0:8].values

X_2019 = df_2019.iloc[:,10:].values
y_2019 = df_2019.iloc[:,9].values
info_2019 = df_2019.iloc[:,0:8].values

In [232]:
df_model.iloc[:10:]

,date,game,score,season,id,home,away,score_home,score_away,result,win_R5_home,draw_R5_home,lost_R5_home,H2H_W,H2H_D,H2H_L,scored_avg_R5_home,conceded_avg_R5_home,points_cum_prev_avg_home,points_cum_h_a_prev_avg_home,win_R5_away,draw_R5_away,lost_R5_away,scored_avg_R5_away,conceded_avg_R5_away,points_cum_prev_avg_away,points_cum_h_a_prev_avg_away
0,2015-09-15,trenčín-zvolen,2-3,2015/2016,83440,trenčín,zvolen,2,3,1,1.000000,0.000000,0.000000,0.0,0.0,0.0,3.500000,1.500000,3.000000,3.000000,0.500000,0.000000,0.500000,4.000000,3.000000,1.500000,0.000000
1,2015-09-15,martin-žilina,2-1,2015/2016,83442,martin,žilina,2,1,1,0.500000,0.000000,0.500000,0.0,0.0,0.0,2.000000,1.500000,1.500000,3.000000,0.000000,0.500000,0.500000,3.000000,4.500000,0.500000,0.500000
2,2015-09-15,banská bystrica-piešťany,2-1,2015/2016,83443,banská bystrica,piešťany,2,1,0,1.000000,0.000000,0.000000,0.0,0.0,0.0,2.500000,0.500000,3.000000,3.000000,1.000000,0.000000,0.000000,4.500000,2.500000,3.000000,3.000000
3,2015-09-18,zvolen-košice,4-2,2015/2016,83445,zvolen,košice,4,2,0,0.333333,0.333333,0.333333,0.0,0.0,0.0,3.666667,2.666667,1.666667,3.000000,0.000000,0.333333,0.666667,2.333333,2.666667,0.666667,0.000000
4,2015-09-18,piešťany-nitra,1-4,2015/2016,83446,piešťany,nitra,1,4,2,0.666667,0.000000,0.333333,0.0,0.0,0.0,3.333333,2.333333,2.000000,3.000000,0.333333,0.000000,0.666667,2.333333,4.000000,1.000000,0.000000
5,2015-09-18,martin-banská bystrica,2-1,2015/2016,83447,martin,banská bystrica,2,1,1,0.333333,0.333333,0.333333,0.0,0.0,0.0,2.000000,1.333333,1.666667,2.500000,1.000000,0.000000,0.000000,2.333333,0.666667,3.000000,3.000000
6,2015-09-18,poprad-žilina,3-2,2015/2016,83448,poprad,žilina,3,2,0,0.333333,0.333333,0.333333,0.0,0.0,0.0,3.000000,3.000000,1.666667,1.500000,0.000000,0.666667,0.333333,2.333333,3.666667,0.666667,0.666667
7,2015-09-18,trenčín-skalica,7-2,2015/2016,83449,trenčín,skalica,7,2,0,0.666667,0.333333,0.000000,0.0,0.0,0.0,3.000000,2.000000,2.333333,2.000000,0.000000,0.333333,0.666667,2.000000,4.000000,0.333333,0.000000
8,2015-09-20,skalica-zvolen,1-9,2015/2016,83450,skalica,zvolen,1,9,2,0.000000,0.250000,0.750000,0.0,0.0,0.0,2.000000,4.750000,0.250000,0.500000,0.500000,0.250000,0.250000,3.750000,2.500000,2.000000,1.000000
9,2015-09-20,trenčín-žilina,3-0,2015/2016,83451,trenčín,žilina,3,0,0,0.750000,0.250000,0.000000,0.0,0.0,0.0,4.000000,2.000000,2.500000,2.333333,0.000000,0.500000,0.500000,2.250000,3.500000,0.500000,0.500000


In [234]:
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test, info_train, info_test  = train_test_split(X, y, info, 
                                                                            test_size = 0.2, stratify = y, random_state = 21)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_2019 = scaler.transform(X_2019)

In [239]:
from sklearn.ensemble import RandomForestClassifier
import pickle

# Fitting Random Forest Classification to the Training set
rf = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 42)
rf.fit(X_train, y_train)

# save the model to disk
pickle.dump(rf, open("my_model.h5", 'wb'))


In [241]:
from sklearn.metrics import classification_report

y_pred_rf = rf.predict(X_test)
y_pred_rf_prob = rf.predict_proba(X_test)

print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.54      0.78      0.64       103
           1       0.25      0.05      0.08        41
           2       0.42      0.34      0.38        62

    accuracy                           0.50       206
   macro avg       0.40      0.39      0.36       206
weighted avg       0.45      0.50      0.45       206



In [243]:
y_pred2019_rf = rf.predict(X_2019)
y_pred2019_rf_prob = rf.predict_proba(X_2019)

df_2019 = pd.DataFrame(info_2019)

df_2019.columns = ['date', 'game', 'score', 'season', 'id', 'home', 'away','score_home']

# Set the outcome variables and probabilities
df_2019['actual'] = y_2019
df_2019['predicted'] = y_pred2019_rf
df_2019['prob_home'] = y_pred2019_rf_prob[:,0]
df_2019['prob_draw'] = y_pred2019_rf_prob[:,1]
df_2019['prob_away'] = y_pred2019_rf_prob[:,2]

# Map back sign outcome
mapping = {0:'home', 1:'draw', 2:'away'}
df_2019 = df_2019.replace({'actual': mapping, 'predicted': mapping})

df_2019['correct']=0
df_2019.loc[df_2019['actual']==df_2019['predicted'],'correct']=1

# Check mean values by sign
print("Correct ratio by prediction")
df_2019.groupby(['predicted'])['correct'].mean()

Correct ratio by prediction


predicted
away    0.500000
draw    0.157895
home    0.546392
Name: correct, dtype: float64